In [1]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("hugging face key")
user = login(token=secret_value_0)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## **Defining the text**


In [2]:
text = """Dear WidgetCorp,
I am writing to express my deep disappointment with your service regarding my recent order (Order
#98765). I had ordered the "SuperWidget" as a birthday gift for my daughter, Emily, whose birthday
was on March 15th. Despite your website promising delivery within three days, the product arrived
only yesterday, missing her birthday entirely.
I attempted to contact your customer service multiple times for an explanation and to expedite the
delivery, but to no avail. My calls and emails went unanswered, leaving me in a difficult position with
no gift for Emily’s special day.
This experience has left a sour taste, and I am reconsidering future purchases from WidgetCorp. I
expect a prompt explanation and an assurance that such delays will not occur in the future.
Sincerely,
Jane Doe

..."""


In [3]:
# # Install required Dependencies
!pip install transformers accelerate tiktoken einops scipy transformers_stream_generator peft deepspeed bitsandbytes trl
!pip install auto-gptq optimum


  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ | / - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.6 MB/s eta 0:00:00
  Created wheel for transformers_stream_generator: filename=transformers_stream_generator-0.0.5-py3-none-any.whl size=12425 sha256=592785923bf7901139616d5a3590355dede0c55f2fd01e4bf94e0f55af15a27e
  Stored in directory: /root/.cache/pip/wheels/95/4a/90/140f

# **Llama 3.1 8B Instruct Q8 by Meta**


In [4]:
# Import required libraries
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
import torch
import warnings
warnings.filterwarnings("ignore")
import textwrap

In [5]:
# Initializing Llama 8B Parameters and Int-16 Character size
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3.1-8B-Instruct", 
    device_map="auto",
    quantization_config= BitsAndBytesConfig(
        load_in_8bit=True,  # Use 8-bit quantization
        ),
    trust_remote_code=True, 
)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")

pipe = pipeline(
    "text-generation",
    model=model,
    model_kwargs={"torch_dtype": torch.bfloat16},
    tokenizer=tokenizer,
)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

### Summerizaition of main points by Llama

In [6]:
prompt = f"""Summarize the following text by listing the key points:
Text: 
```{text}```
"""
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": prompt},
]

output = pipe(messages, max_new_tokens=256, pad_token_id=128001)  
print(output[0]['generated_text'][-1]['content'])

Here are the key points from the text:

1. The customer, Jane Doe, is writing to express disappointment with the service she received from WidgetCorp.
2. The issue is related to a delayed delivery of the "SuperWidget" (Order #98765) which was supposed to arrive within 3 days.
3. The product arrived late, missing the birthday of her daughter Emily (March 15th).
4. Jane attempted to contact customer service multiple times, but her calls and emails went unanswered.
5. The customer is considering not making future purchases from WidgetCorp due to this experience.
6. Jane is expecting a prompt explanation and assurance that similar delays will not occur in the future.


### Question Answering by Llama

In [7]:
prompt = f"""Can you provide the answers to the following questions, and format them as "Answer 1: ..." and "Answer 2: ..."?
Questions:
1. What was the reason for the customer's disappointment with WidgetCorp?
2. How did WidgetCorp's customer service respond to the customer's inquiries about the late delivery?

Text: 
```{text}```
"""
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": prompt},
]

output = pipe(messages, max_new_tokens=256, pad_token_id=128001)  
print(output[0]['generated_text'][-1]['content'])

Answer 1: The customer's disappointment with WidgetCorp was due to the late delivery of the "SuperWidget" order, which missed the birthday of the customer's daughter, Emily, on March 15th.

Answer 2: WidgetCorp's customer service did not respond to the customer's inquiries about the late delivery, as the customer's calls and emails went unanswered, leaving them in a difficult position.


### NER by Llama

In [8]:
prompt = f"""Please identify all the named entities from the text delimited by triple backticks, and categorize them 

Provide the output in a structured format like this:
1. Entity: [Entity Name], Type: [Entity Type]
2. Entity: [Entity Name], Type: [Entity Type]
...


Text: 
```{text}```
"""
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": prompt},
]

output = pipe(messages, max_new_tokens=256,  pad_token_id=128001)
print(output[0]['generated_text'][-1]['content'])

1. Entity: WidgetCorp, Type: Organization
2. Entity: Emily, Type: Person
3. Entity: Jane Doe, Type: Person
4. Entity: March 15th, Type: Date
5. Entity: SuperWidget, Type: Product
6. Entity: Order #98765, Type: Identifier (Order number)


### Translation by Llama

In [9]:


prompt = f"""Can you translate the following text delimited by triple backticks from English to French.
Text: 
```{text}```
"""
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": prompt},
]

# Translate from English to French
output = pipe(messages, max_new_tokens=256, pad_token_id=128001)
french_translation = output[0]['generated_text'][-1]['content']

# Print the French translation
print("French Translation:")
print(french_translation)

French Translation:
Voici la traduction du texte en français :

```Cher WidgetCorp,

Je vous écris pour exprimer ma profonde déception concernant votre service concernant ma commande récente (Commande #98765). J'avais commandé le "SuperWidget" en tant que cadeau d'anniversaire pour ma fille, Emily, dont l'anniversaire était le 15 mars. Malgré la promesse de votre site Web de livraison dans les trois jours, le produit n'est arrivé que hier, manquant ainsi son anniversaire entièrement.

J'ai tenté de contacter votre service client à plusieurs reprises pour une explication et pour accélérer la livraison, mais sans succès. Mes appels et courriels n'ont pas été répondu, laissant mon époux et moi dans une situation difficile sans cadeau pour l'anniversaire spécial d'Emily.

Cette expérience a laissé un goût amer, et je suis en train de réfléchir à des achats futurs de la part de WidgetCorp. Je m'attends à une explication rapide et à une assurance que de telles retards ne se produiront pas à 

# Mistral-7B-Instruct-v0.3

In [10]:
# Import required libraries
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
import warnings
warnings.filterwarnings("ignore")
import textwrap
import torch

In [11]:
# Initializing Llama 8B Parameters and Int-16 Character size
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3", 
    device_map="auto",
#     quantization_config= BitsAndBytesConfig(
#         load_in_8bit=True,  # Use 8-bit quantization
#         ),
    trust_remote_code=True, 
)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")

pipe = pipeline(
    "text-generation",
    model=model,
    model_kwargs={"torch_dtype": torch.bfloat16},
    tokenizer=tokenizer,
)

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

### Summerizaition of main points by Mistral


In [12]:
prompt = f"""Summarize the following text by listing the key points:
Text: 
```{text}```
"""
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": prompt},
]

output = pipe(messages, max_new_tokens=256, pad_token_id=128001)  
print(output[0]['generated_text'][-1]['content'])

 - Customer (Jane Doe) is expressing disappointment about a late delivery of the "SuperWidget" order (#98765) from WidgetCorp.
- The order was intended as a birthday gift for her daughter Emily, whose birthday was on March 15th.
- The product arrived a day late, missing Emily's birthday.
- The customer attempted to contact WidgetCorp's customer service multiple times for an explanation and to expedite the delivery, but received no response.
- The lack of communication and the missed delivery has left a negative impression, and the customer is reconsidering future purchases from WidgetCorp.
- The customer expects a prompt explanation and assurance that such delays will not occur in the future.


### Question Answering by Mistral

In [13]:
prompt = f"""Can you provide the answers to the following questions, and format them as "Answer 1: ..." and "Answer 2: ..."?
Questions:
1. What was the reason for the customer's disappointment with WidgetCorp?
2. How did WidgetCorp's customer service respond to the customer's inquiries about the late delivery?

Text: 
```{text}```
"""
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": prompt},
]

output = pipe(messages, max_new_tokens=256, pad_token_id=128001)  
print(output[0]['generated_text'][-1]['content'])

 Answer 1: The reason for the customer's disappointment with WidgetCorp was that the delivery of her order (Order #98765) for the "SuperWidget" as a birthday gift for her daughter, Emily, arrived late and missed Emily’s birthday.

Answer 2: WidgetCorp's customer service did not respond adequately to the customer's inquiries about the late delivery. The customer attempted to contact customer service multiple times for an explanation and to expedite the delivery, but their calls and emails went unanswered.


### NER by Mistral

In [14]:
prompt = f"""Please identify all the named entities from the text delimited by triple backticks, and categorize them 

Provide the output in a structured format like this:
1. Entity: [Entity Name], Type: [Entity Type]
2. Entity: [Entity Name], Type: [Entity Type]
...


Text: 
```{text}```
"""
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": prompt},
]

output = pipe(messages, max_new_tokens=256,  pad_token_id=128001)
print(output[0]['generated_text'][-1]['content'])

 1. Entity: WidgetCorp, Type: Organization
2. Entity: Order #98765, Type: Product Order
3. Entity: SuperWidget, Type: Product
4. Entity: Emily, Type: Person
5. Entity: March 15th, Type: Date
6. Entity: Customer Service, Type: Department
7. Entity: Jane Doe, Type: Person


### Translation by Mistral

In [15]:


prompt = f"""Can you translate the following text delimited by triple backticks from English to French.
Text: 
```{text}```
"""
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": prompt},
]

# Translate from English to French
output = pipe(messages, max_new_tokens=256, pad_token_id=128001)
french_translation = output[0]['generated_text'][-1]['content']

# Print the French translation
print("French Translation:")
print(french_translation)

French Translation:
 Voici la traduction du texte en français :

```Chère WidgetCorp,
Je vous écris pour exprimer ma profonde déception envers votre service concernant ma dernière commande (Commande #98765). J'avais commandé le "SuperWidget" en tant que cadeau de naissance pour ma fille, Emily, dont le jour de naissance était le 15 mars. Malgré ce que votre site web promettait une livraison en trois jours, le produit n'est arrivé qu'hier, manquant complètement son jour de naissance.
J'ai tenté de contacter votre service client multiple fois pour obtenir une explication et pour accélérer la livraison, mais sans succès. Mes appels et mes courriels ont été ignorés, laissant moi dans une position difficile sans cadeau pour le jour spécial d'Emily.
Cette expérience a laissé un goût amer, et je reconsidère les achats futurs de WidgetCorp. Je vous attend une explication rapide et


#  Phi 3.5 Mini Instruct by Microsoft¶


In [16]:
# Import required libraries
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import warnings
warnings.filterwarnings("ignore")
import torch

In [17]:
torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct", 
    device_map="auto", 
    torch_dtype="auto", 
    trust_remote_code=True, 
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

### Text Summarization by Phi


In [18]:
prompt = f"""Summarize the following text by listing the key points:
Text: 
```{text}```
"""
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": prompt},
]

output = pipe(messages, **generation_args)
print(output[0]['generated_text'])

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


 - Jane Doe expresses deep disappointment with WidgetCorp for late delivery of "SuperWidget."

- Order #98765 was intended as a birthday gift for her daughter, Emily, but arrived after her birthday on March 15th.

- WidgetCorp's website promised delivery within three days, which was not met.

- Jane Doe attempted to contact customer service for an explanation and expedited delivery but received no response.

- The delay has negatively impacted her experience, and she is reconsidering future purchases from WidgetCorp.

- Jane Doe expects a prompt explanation and assurance that future delays will be avoided.


### Question Answering by Phi


In [19]:
prompt = f"""Can you provide the answers to the following questions, and format them as "Answer 1: ..." and "Answer 2: ..."?
Questions:
1. What was the reason for the customer's disappointment with WidgetCorp?
2. How did WidgetCorp's customer service respond to the customer's inquiries about the late delivery?

Text: 
```{text}```
"""
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": prompt},
]

output = pipe(messages, max_new_tokens=256, pad_token_id=128001)  
print(output[0]['generated_text'][-1]['content'])

 Answer 1: The customer's disappointment with WidgetCorp was due to the late delivery of the "SuperWidget," which arrived after their daughter Emily's birthday, missing the occasion entirely.

Answer 2: WidgetCorp's customer service did not respond to the customer's inquiries about the late delivery, as the customer's calls and emails went unanswered, leaving them without a resolution or explanation for the delay.


### NER by PHI

In [20]:
prompt = f"""Please identify the named entities from the text delimited by triple backticks, and categorize them

Provide the output in a structured format like this:
1. Entity: [Entity Name], Type: [Entity Type]
2. Entity: [Entity Name], Type: [Entity Type]
...


Text: 
```{text}```
"""
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": prompt},
]

output = pipe(messages, max_new_tokens=256,  pad_token_id=128001)
print(output[0]['generated_text'][-1]['content'])

 1. Entity: WidgetCorp, Type: Organization
2. Entity: Order #98765, Type: Order Reference
3. Entity: SuperWidget, Type: Product
4. Entity: Emily, Type: Person (Recipient of the product)
5. Entity: March 15th, Type: Date
6. Entity: Jane Doe, Type: Person (Sender of the letter)
7. Entity: Customer Service, Type: Department/Service

Note: The date and person names are categorized as entities because they represent specific individuals or time points that are relevant to the context of the text. The organization and product are entities as they represent the business and the item involved in the transaction. The order reference is an entity as it is a specific identifier for the transaction in question. The department/service is an entity as it represents a part of the organization that is being interacted with.


### Translation by Phi

In [21]:
prompt = f"""Can you translate the following text delimited by triple backticks from English to French.
Text: 
```{text}```
"""
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": prompt},
]

# Translate from English to French
output = pipe(messages, max_new_tokens=256, pad_token_id=128001)
french_translation = output[0]['generated_text'][-1]['content']

# Print the French translation
print("French Translation:")
print(french_translation)

French Translation:
 Cher WidgetCorp,

Je vous écris pour exprimer mon profond désappointement concernant le service concernant mon récent commande (Commande #98765). J'avais commandé le "SuperWidget" comme cadeau de naissance pour ma fille, Emily, dont l'anniversaire était le 15 mars. Malgré votre site web promettant une livraison dans trois jours, le produit est arrivé seulement hier, manquant totalement son anniversaire.

J'ai tenté de contacter votre service client plusieurs fois pour obtenir une explication et accélérer la livraison, mais sans succès. Mes appels et emails restaient sans réponse, me laissant dans une situation difficile sans cadeau pour l'anniversaire spécial d'Emily.

Cette expérience a laissé un goût amer, et je réévalue les futures achats chez WidgetCorp. Je m'attends à une explication rapide et une assurance que de telles retards ne se reproduiront plus à l'avenir.

Cordialement,
Jane Do


# Qwen 1.8B Chat Q4 by Alibaba

In [22]:
# Import required libraries
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import warnings
warnings.filterwarnings("ignore")
import textwrap

In [23]:
# Initializing Qwen 1.8B Parameters and Int-4 Character size
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-1_8B-Chat-Int4", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen-1_8B-Chat-Int4",
    device_map="auto",
    trust_remote_code=True
).eval()

tokenizer_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

tokenization_qwen.py:   0%|          | 0.00/9.62k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Qwen/Qwen-1_8B-Chat-Int4:
- tokenization_qwen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


qwen.tiktoken:   0%|          | 0.00/2.56M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

...

cpp_kernels.py:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Qwen/Qwen-1_8B-Chat-Int4:
- cpp_kernels.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/Qwen/Qwen-1_8B-Chat-Int4:
- modeling_qwen.py
- qwen_generation_utils.py
- cpp_kernels.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

Some weights of the model checkpoint at Qwen/Qwen-1_8B-Chat-Int4 were not used when initializing QWenLMHeadModel: ['transformer.h.0.attn.c_proj.bias', 'transformer.h.0.mlp.c_proj.bias', 'transformer.h.0.mlp.w1.bias', 'transformer.h.0.mlp.w2.bias', 'transformer.h.1.attn.c_proj.bias', 'transformer.h.1.mlp.c_proj.bias', 'transformer.h.1.mlp.w1.bias', 'transformer.h.1.mlp.w2.bias', 'transformer.h.10.attn.c_proj.bias', 'transformer.h.10.mlp.c_proj.bias', 'transformer.h.10.mlp.w1.bias', 'transformer.h.10.mlp.w2.bias', 'transformer.h.11.attn.c_proj.bias', 'transformer.h.11.mlp.c_proj.bias', 'transformer.h.11.mlp.w1.bias', 'transformer.h.11.mlp.w2.bias', 'transformer.h.12.attn.c_proj.bias', 'transformer.h.12.mlp.c_proj.bias', 'transformer.h.12.mlp.w1.bias', 'transformer.h.12.mlp.w2.bias', 'transformer.h.13.attn.c_proj.bias', 'transformer.h.13.mlp.c_proj.bias', 'transformer.h.13.mlp.w1.bias', 'transformer.h.13.mlp.w2.bias', 'transformer.h.14.attn.c_proj.bias', 'transformer.h.14.mlp.c_proj.bias'

generation_config.json:   0%|          | 0.00/249 [00:00<?, ?B/s]

## Text Summarization by Quen


In [24]:
prompt = f"""Summarize the following text delimited by triple backticks.
Text: 
```{text}```
"""

response, history = model.chat(tokenizer, prompt, history=None)
print(textwrap.fill(response, width=150))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Dear WidgetCorp,  I am writing to express my deep disappointment with your service regarding my recent order (Order #98765). I had ordered the
"SuperWidget" as a birthday gift for my daughter, Emily, whose birthday was on March 15th. Despite your website promises delivery within three days,
the product arrived only yesterday, missing her birthday entirely.  I attempted to contact your customer service multiple times for an explanation and
to expedite the delivery, but to no avail. My calls and emails went unanswered, leaving me in a difficult position with no gift for Emily’s special
day.  This experience has left a sour taste, and I am reconsidering future purchases from WidgetCorp. I expect a prompt explanation and an assurance
that such delays will not occur in the future.  Sincerely,  Jane Doe  ...


### Question Answering by Quen


In [25]:
prompt = f"""Can you provide the answers to the following questions, and format them as "Answer 1: ..." and "Answer 2: ..."?
Questions:
1. What was the reason for the customer's disappointment with WidgetCorp?
2. How did WidgetCorp's customer service respond to the customer's inquiries about the late delivery?

Text: 
```{text}```
"""
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": prompt},
]

output = pipe(messages, max_new_tokens=256, pad_token_id=128001)  
print(output[0]['generated_text'][-1]['content'])

 Answer 1: The customer's disappointment with WidgetCorp was due to the late delivery of the "SuperWidget," which arrived after their daughter Emily's birthday, missing the occasion entirely.

Answer 2: WidgetCorp's customer service did not respond to the customer's inquiries about the late delivery, as the customer's calls and emails went unanswered, leaving them without a resolution or explanation for the delay.


### NER by Quen


In [26]:
prompt = f"""Please identify the named entities from the text delimited by triple backticks, and categorize them

Provide the output in a structured format like this:
1. Entity: [Entity Name], Type: [Entity Type]
2. Entity: [Entity Name], Type: [Entity Type]
...


Text: 
```{text}```
"""
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": prompt},
]

output = pipe(messages, max_new_tokens=256,  pad_token_id=128001)
print(output[0]['generated_text'][-1]['content'])

 1. Entity: WidgetCorp, Type: Organization
2. Entity: Order #98765, Type: Order Reference
3. Entity: SuperWidget, Type: Product
4. Entity: Emily, Type: Person (Recipient of the product)
5. Entity: March 15th, Type: Date
6. Entity: Jane Doe, Type: Person (Sender of the letter)
7. Entity: Customer Service, Type: Department/Service

Note: The date and person names are categorized as entities because they represent specific individuals or time points that are relevant to the context of the text. The organization and product are entities as they represent the business and the item involved in the transaction. The order reference is an entity as it is a specific identifier for the transaction in question. The department/service is an entity as it represents a part of the organization that is being interacted with.


### Translation by Quen


In [27]:
prompt = f"""Can you translate the following text delimited by triple backticks from English to French.
Text: 
```{text}```
"""
messages = [
    {"role": "system", "content": "You are a helpful AI assistant."},
    {"role": "user", "content": prompt},
]

# Translate from English to French
output = pipe(messages, max_new_tokens=256, pad_token_id=128001)
french_translation = output[0]['generated_text'][-1]['content']

# Print the French translation
print("French Translation:")
print(french_translation)

French Translation:
 Cher WidgetCorp,

Je vous écris pour exprimer mon profond désappointement concernant le service concernant mon récent commande (Commande #98765). J'avais commandé le "SuperWidget" comme cadeau de naissance pour ma fille, Emily, dont l'anniversaire était le 15 mars. Malgré votre site web promettant une livraison dans trois jours, le produit est arrivé seulement hier, manquant totalement son anniversaire.

J'ai tenté de contacter votre service client plusieurs fois pour obtenir une explication et accélérer la livraison, mais sans succès. Mes appels et emails restaient sans réponse, me laissant dans une situation difficile sans cadeau pour l'anniversaire spécial d'Emily.

Cette expérience a laissé un goût amer, et je réévalue les futures achats chez WidgetCorp. Je m'attends à une explication rapide et une assurance que de telles retards ne se reproduiront plus à l'avenir.

Cordialement,
Jane Do
